## BRONZE TO SILVER LAYER

### Silver Layer - Stock Events History


In [1]:
# Import necessary libraries and utility functions
import pandas as pd
from ETLTools.globalpath import GlobalPath
from ETLTools.utilities import replace_punctuation_from_columns

In [2]:
# Instantiate GlobalPath


stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockevents_silver_file_path = GlobalPath(
    "DATA/SILVER/StockEvents/StockEvents_data.csv"
)

### Data Processing

This section handles the collection of data from the Bronze layer.

- Files are checked for availability.
- Data is read from each file, processed, and stored in a list of DataFrames.
- Process stock event data from the Bronze layer and transform it for the Silver layer.


In [3]:
# Initialize an empty list to store DataFrames
df_stock_events_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = stockdata_bronze_layer_path.check_files_availability(
    file_pattern="*.csv"
)

# Process each CSV file found in the Bronze layer
for file_path in file_paths:
    print(f"Processing file: {file_path.relative_path()}")

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Extract stock name from file path and add it as a column
    df["symbol"] = file_path.name.split("_")[0].upper().strip()

    # Append the DataFrame to the list
    df_stock_events_list.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(df_stock_events_list, ignore_index=True)

# Harmonize column names to ensure consistency
df = replace_punctuation_from_columns(df)

# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

Number of Files Detected: 216
Processing file: DATA/BRONZE/StockData/BHAGERIA_2020_05.csv
Processing file: DATA/BRONZE/StockData/BHAGERIA_2020_06.csv
Processing file: DATA/BRONZE/StockData/BPCL_2021_08.csv
Processing file: DATA/BRONZE/StockData/BPCL_2021_09.csv
Processing file: DATA/BRONZE/StockData/BPCL_2021_10.csv
Processing file: DATA/BRONZE/StockData/BPCL_2021_11.csv
Processing file: DATA/BRONZE/StockData/BPCL_2021_12.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_01.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_02.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_03.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_04.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_05.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_06.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_07.csv
Processing file: DATA/BRONZE/StockData/BPCL_2022_08.csv
Processing file: DATA/BRONZE/StockData/GOLDBEES_2021_02.csv
Processing file: DATA/BRONZE/StockData/GOLDBEES_2021_03.csv
Pr

Processing file: DATA/BRONZE/StockData/TATACHEM_2022_05.csv
Processing file: DATA/BRONZE/StockData/TATACHEM_2022_06.csv
Processing file: DATA/BRONZE/StockData/TATACHEM_2022_07.csv
Processing file: DATA/BRONZE/StockData/TATACHEM_2022_08.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_04.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_05.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_06.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_07.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_08.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_09.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_10.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_11.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2020_12.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2021_01.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2021_02.csv
Processing file: DATA/BRONZE/StockData/TATAMOTORS_2021_03.csv
Processing file:

Processing file: DATA/BRONZE/StockData/TATAPOWER_2022_11.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2022_12.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_01.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_02.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_03.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_04.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_05.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_06.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_07.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_08.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_09.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_10.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_11.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2023_12.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2024_01.csv
Processing file: DATA/BRONZE/StockData/TATAPOWER_2024_02.csv
Processing file: DATA/BR

In [4]:
# Convert datetime strings to date objects for consistency
df["date"] = pd.to_datetime(df["date"]).dt.date

# Pivot the DataFrame: columns become rows
df = df.melt(
    id_vars=["date", "symbol"],
    value_vars=["dividends", "stock_splits", "capital_gains"],
    var_name="event",
    value_name="value",
)

# Convert event names to uppercase for uniformity
df["event"] = df["event"].str.upper()

# Round numerical values to 2 decimal places
df["value"] = df["value"].fillna(0)
df["value"] = df["value"].round(2)

# Filter out rows where the value is zero or less than a threshold
df = df[df["value"] > 0]

# Sort the DataFrame by stock name, date, and event for easier analysis
df = df.sort_values(by=["symbol", "date", "event"])

# Select relevant columns to export
df = df[["date", "symbol", "event", "value"]]

# Save the cleaned and transformed DataFrame as a CSV file in the Silver layer
df.to_csv(stockevents_silver_file_path, index=False)
print("SILVER Layer CSV file for Stock Events history successfully created at:")
print(stockevents_silver_file_path.relative_path())

# Log the DataFrame debug for verification
df.info()

SILVER Layer CSV file for Stock Events history successfully created at:
DATA/SILVER/StockEvents/StockEvents_data.csv
<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 69 to 3847
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    18 non-null     object 
 1   symbol  18 non-null     object 
 2   event   18 non-null     object 
 3   value   18 non-null     float64
dtypes: float64(1), object(3)
memory usage: 720.0+ bytes
